In [5]:
import subprocess
import numpy as np

In [6]:
# Easy access/variable params
intro_time = 264 #22 years (2022 intro, sort of)
sim_duration = 864 #72 years

# needed for other things, but not tuneable params
n_init_rxtrs = 100
n_assem_core = 3
assem_size = 29565
init_total_assem_size = n_init_rxtrs * n_assem_core * assem_size
cycle_time = 18

In [41]:
init_total_assem_size

8869500

# Protoype Definitions:
1. LWRs : Full additive availability and ramped additive availability
2. Fuel Cycle Facilities

## 1. LWR

In [54]:
# LWR prototype for full additive availability upon introduction
lwr_full = {'name' : 'LWR',
            'lifetime' : 720,
            'config' : {'Reactor' : {'fuel_incommods' :  {'val' : ['UOX_Additive', 'UOX_NoAdditive', 'InitUOX']},
                                     'fuel_outcommods' : {'val' : ['SpentUOX_Additive', 'SpentUOX_NoAdditive', 'SpentUOX_NoAdditive']},
                                     'fuel_inrecipes' :  {'val' : ['UOX_232', 'UOX_no232', 'UOX_no232']},
                                     'fuel_outrecipes' : {'val' : ['SpentUOX_232', 'SpentUOX_no232', 'SpentUOX_no232']},
                                     'fuel_prefs' : {'val' : [1, 2, 2.5]},
                                     'pref_change_times' : {'val' : intro_time},
                                     'pref_change_commods' : {'val' : 'UOX_Additive'},
                                     'pref_change_values' : {'val' : 3},
                                     'cycle_time' : cycle_time,
                                     'refuel_time' : 0,
                                     'assem_size' : assem_size,
                                     'n_assem_core' : n_assem_core,
                                     'n_assem_batch' : 1,
                                     'power_cap' : 1000
                                    }
                       }
           }

# LWR prototype for partial additive availability/slow utility uptake upon introduction
intro_50 = intro_time + 3 * cycle_time
intro_100 = intro_50 + 3 * cycle_time
lwr_ramp = {'name' : 'LWR',
            'lifetime' : 720,
            'config' : {'Reactor' : {'fuel_incommods' :  {'val' : ['UOX_Additive', 'UOX_50pctAdditive', 'UOX_10pctAdditive', 'UOX_NoAdditive', 'InitUOX']},
                                     'fuel_outcommods' : {'val' : ['SpentUOX_Additive', 'SpentUOX_50pctAdditive', 'SpentUOX_10pctAdditive', 'SpentUOX_NoAdditive', 'SpentUOX_NoAdditive']},
                                     'fuel_inrecipes' :  {'val' : ['UOX_232', 'UOX_50pct232', 'UOX_10pct232', 'UOX_no232', 'UOX_no232']},
                                     'fuel_outrecipes' : {'val' : ['SpentUOX_232', 'SpentUOX_50pct232', 'SpentUOX_10pct232', 'SpentUOX_no232', 'SpentUOX_no232']},
                                     'fuel_prefs' : {'val' : [1, 1, 1, 2, 2.5]},
                                     'pref_change_times' : {'val' : [intro_time, intro_50, intro_100]},
                                     'pref_change_commods' : {'val' : ['UOX_10pctAdditive', 'UOX_50pctAdditive', 'UOX_Additive']},
                                     'pref_change_values' : {'val' : [3, 4, 5]},
                                     'cycle_time' : cycle_time,
                                     'refuel_time' : 0,
                                     'assem_size' : assem_size,
                                     'n_assem_core' : n_assem_core,
                                     'n_assem_batch' : 1,
                                     'power_cap' : 1000
                                    }
                       }
           }

## 2. Fuel Cycle Facilities

In [53]:
init_uox_source = {'name' : 'SourceInitUOX', 
                   'config' : {'Source' : {'outcommod' : 'InitUOX',
                                           'outrecipe' : 'UOX_no232',
                                           'inventory_size' : init_total_assem_size
                                          }
                              }
                  }
from facilities import natu_source, non_source, add_source, enrich, du_store, store_no232, mix_no232, store_232, mix_232, lwr_cool, lwr_store, sink

In [55]:
# mixer scheme for 10, 50 pct additive UOX streams

# first need extra storage to request separate stream/recipe
store_pct_no232 = {'name' : 'StorageRampNoAdditive', 
                   'config' : {'Storage' : {'in_commods' : {'val' : 'UOX_NoAdditive'}, 
                                            'in_recipe' : 'UOX_no232', 
                                            'out_commods' : {'val' : 'Mixer_UOX_NoAdditive'}, 
                                            'residence_time' : 0,
                                            'throughput' : 1e10,
                                            'max_inv_size' : 1e20
                                           }
                              }
                   }
store_pct_232 = {'name' : 'StorageRampAdditive', 
                 'config' : {'Storage' : {'in_commods' : {'val' : 'UOX_Additive'}, 
                                          'in_recipe' : 'UOX_232', 
                                          'out_commods' : {'val' : 'Mixer_UOX_Additive'}, 
                                          'residence_time' : 0,
                                          'throughput' : 1e10,
                                          'max_inv_size' : 1e20
                                         }
                            }
                }

# 50 pct additive material stream
mix_non_stream50 = {'info' : {'mixing_ratio' : 0.5, 
                              'buf_size' : 1e8
                             },
                    'commodities' : {'item' : {'commodity' : 'Mixer_UOX_NoAdditive', 
                                               'pref' : 1.0
                                              }
                                    }
                   }
mix_add_stream50 = {'info' : {'mixing_ratio' : 0.5,
                              'buf_size' : 1e8
                             },
                    'commodities' : {'item' : {'commodity' : 'Mixer_UOX_Additive',
                                               'pref' : 1.0
                                              }
                                    }
                   }
mix_50pct232 = {'name' : 'Mixer50pctAdditive',
                'config' : {'Mixer' : {'in_streams' : {'stream' : [mix_non_stream50, mix_add_stream50]},
                                       'out_commod' : 'UOX_50pctAdditive',
                                       'throughput' : 1e10,
                                       'out_buf_size' : 1e20
                                      }
                           }
               }

# 10 pct additive material stream
mix_non_stream10 = {'info' : {'mixing_ratio' : 0.9, 
                              'buf_size' : 1e8
                             },
                    'commodities' : {'item' : {'commodity' : 'Mixer_UOX_NoAdditive', 
                                               'pref' : 1.0
                                              }
                                    }
                   }
mix_add_stream10 = {'info' : {'mixing_ratio' : 0.1,
                              'buf_size' : 1e8
                             },
                    'commodities' : {'item' : {'commodity' : 'Mixer_UOX_Additive',
                                               'pref' : 1.0
                                              }
                                    }
                   }
mix_10pct232 = {'name' : 'Mixer10pctAdditive',
                'config' : {'Mixer' : {'in_streams' : {'stream' : [mix_non_stream10, mix_add_stream10]},
                                       'out_commod' : 'UOX_10pctAdditive',
                                       'throughput' : 1e10,
                                       'out_buf_size' : 1e20
                                      }
                           }
               }

# Regions and Institutions:
1. Deploy Inst (Init Fleet)
2. Manager Inst (Remainder of FC facilities)
3. Growth Region

## 1. Init LWR Fleet (Deploy Inst)

In [10]:
init_lwr_prototypes = ['LWR' for x in range(0, n_init_rxtrs)]
n_builds = [1 for x in range(0, n_init_rxtrs)]
build_times = [x + 1 for x in range(0, 17) for y in range(0,6)]
del build_times[-3:-1]
lifetimes = [181, 186, 191, 196, 
             201, 206, 211, 216, 221, 226, 231, 236, 241, 246, 251, 256, 261, 266, 271, 276, 281, 286, 291, 296,
             301, 306, 311, 316, 321, 326, 331, 336, 341, 346, 351, 356, 361, 366, 371, 376, 381, 386, 391, 396,
             401, 406, 411, 416, 421, 426, 431, 436, 441, 446, 451, 456, 461, 466, 471, 476, 481, 486, 491, 496,
             501, 506, 511, 516, 521, 526, 531, 536, 541, 546, 551, 556, 561, 566, 571, 576, 581, 586, 591, 596,
             601, 606, 611, 616, 621, 626, 631, 636, 641, 646, 651, 656, 661, 666, 671, 676
            ]

assert len(init_lwr_prototypes) == n_init_rxtrs
assert len(n_builds) == n_init_rxtrs
assert len(build_times) == n_init_rxtrs
assert len(lifetimes) == n_init_rxtrs

In [11]:
init_fleet = {'name' : 'InitFleet', 
              'config' : {'DeployInst' : {'prototypes' :  {'val' : init_lwr_prototypes},
                                          'n_build' :     {'val' : n_builds},
                                          'build_times' : {'val' : build_times},
                                          'lifetimes' :   {'val' : lifetimes}
                                         }
                         }
             }

## 2a. FC Inst (Manager Inst)

In [43]:
fc_prototypes = ['SourceNatU', 'SourceNoAdditiveIsos', 'SourceAdditiveIsos', 'Enrichment', 'StorageDepU', 
                 'StorageNoAdditive', 'MixerNoAdditive', 'StorageAdditive', 'MixerAdditive', 
                 'LWR', 'CoolingLWR', 'StorageLWR', 'Waste'
                ]

fc_inst = {'name' : 'FCInst', 
           'initialfacilitylist' : {'entry' : [{'number' : 1, 'prototype' : 'SourceInitUOX'},
                                               {'number' : 1, 'prototype' : 'SourceNatU'},
                                               {'number' : 1, 'prototype' : 'SourceNoAdditiveIsos'},
                                               {'number' : 1, 'prototype' : 'SourceAdditiveIsos'},
                                               {'number' : 1, 'prototype' : 'Enrichment'},
                                               {'number' : 1, 'prototype' : 'StorageDepU'},
                                               {'number' : 1, 'prototype' : 'StorageNoAdditive'},
                                               {'number' : 1, 'prototype' : 'MixerNoAdditive'},
                                               {'number' : 1, 'prototype' : 'StorageAdditive'},
                                               {'number' : 1, 'prototype' : 'MixerAdditive'},
                                               {'number' : 1, 'prototype' : 'CoolingLWR'},
                                               {'number' : 1, 'prototype' : 'StorageLWR'},
                                               {'number' : 1, 'prototype' : 'Waste'}]
                                   },
           'config' : {'ManagerInst' : {'prototypes' : {'val' : fc_prototypes}
                                       }
                      }
          }

## 2b. Add-on Facilities for Additive Ramp-up

In [50]:
ramp_prototypes = ['StorageRampNoAdditive', 'StorageRampAdditive', 
                   'Mixer50pctAdditive', 'Mixer10pctAdditive'
                  ]

t = intro_time

ramp_inst = {'name' : 'RampInst', 
             'config' : {'DeployInst' : {'prototypes' :  {'val' : ramp_prototypes},
                                         'n_build' :     {'val' : [1, 1, 1, 1]},
                                         'build_times' : {'val' : [t, t, t, t]}
                                        }
                        }
            }

## 3. Growth Region

In [45]:
lin_func = {'piece' : [{'start' : 18,
                        'function' : {'type' : 'linear',
                                      'params' : '0 100000'
                                     }
                       }]
           }

growth_region = {'GrowthRegion' : {'growth' : {'item' : [{'commod' : 'power',
                                                          'piecewise_function' : lin_func
                                                         }]
                                              }
                                  }
                }

# Recipes
-  Here: 
  1. Depleted U
  2. Natural U
-  100 ppt Init U232 Additive Recipes (in recipe_100ppt.py):
  1. NonAdditive U Isotopes (U234)
  2. Additive U Isotopes (U232, U233, U234)
  3. Almost UOX NonAdditive Enr Ratio
  4. Almost UOX Additive Enr Ratio
  5. UOX without Additive
  6. UOX with Additive
  7. Spent UOX from #5
  8. Spent UOX from #6

In [46]:
dep_u = {'name' : 'DU',
         'basis' : 'mass',
         'nuclide' : [{'id' : 'U235', 'comp' : 0.0025}, 
                      {'id' : 'U238', 'comp' : 0.9975}]
        }
nat_u = {'name' : 'NU',
         'basis' : 'mass',
         'nuclide' : [{'id' : 'U235', 'comp' : 0.007110}, 
                      {'id' : 'U238', 'comp' : 0.992890}]
        }

In [47]:
# Recipes from 100 ppt U232 additive @ beginning of enrichment
from recipe_100ppt import (isos_no232, isos_232, enr_no232, enr_232, 
                           uox_no232, uox_10pct232, uox_50pct232, uox_232, 
                           spent_no232, spent_10pct232, spent_50pct232, spent_232)

# Main Input File

In [56]:
control = {'duration' : sim_duration, 
           'startmonth' : 1, 
           'startyear' : 2000,
           'dt' : 86400
          }

archetypes = {'spec' : [{'lib' : 'cycamore', 'name' : 'Source'},
                        {'lib' : 'cycamore', 'name' : 'Enrichment'},
                        {'lib' : 'cycamore', 'name' : 'Mixer'},
                        {'lib' : 'cycamore', 'name' : 'Reactor'},
                        {'lib' : 'cycamore', 'name' : 'Storage'},
                        {'lib' : 'cycamore', 'name' : 'Sink'},
                        {'lib' : 'cycamore', 'name' : 'DeployInst'},
                        {'lib' : 'cycamore', 'name' : 'ManagerInst'},
                        {'lib' : 'cycamore', 'name' : 'GrowthRegion'},
                      ]
             }


# full additive availability at intro
full_region = {'name' : 'GrowthRegion', 
               'config' : growth_region, 
               'institution' : [init_fleet, fc_inst]
              }
full_sim = {'simulation' : {'control' : control,
                            'archetypes' : archetypes,
                            'region' : full_region,
                            'facility' : [init_uox_source, natu_source, non_source, add_source, enrich,
                                          du_store, store_no232, mix_no232, store_232, mix_232, 
                                          lwr_full, lwr_cool, lwr_store, sink
                                         ],
                            'recipe' : [dep_u, nat_u, isos_no232, isos_232, enr_no232, enr_232, 
                                        uox_no232, uox_232, spent_no232, spent_232
                                       ]
                           }
           }
# ramp up additive availability at intro
ramp_region = {'name' : 'GrowthRegion', 
               'config' : growth_region, 
               'institution' : [init_fleet, fc_inst, ramp_inst]
              }
ramp_sim = {'simulation' : {'control' : control,
                            'archetypes' : archetypes,
                            'region' : ramp_region,
                            'facility' : [init_uox_source, natu_source, non_source, add_source, enrich,
                                          du_store, store_no232, mix_no232, store_232, mix_232, 
                                          store_pct_no232, store_pct_232, mix_50pct232, mix_10pct232, 
                                          lwr_ramp, lwr_cool, lwr_store, sink
                                         ],
                            'recipe' : [dep_u, nat_u, isos_no232, isos_232, enr_no232, enr_232, 
                                        uox_no232, uox_10pct232, uox_50pct232, uox_232,
                                        spent_no232, spent_10pct232, spent_50pct232, spent_232
                                       ]
                           }
           }

In [57]:
filebase = 'test_py'

in_file = filebase + '.py'
sim_file = '../output/' + filebase + '.sqlite'

with open(in_file, 'w') as file: 
    file.write('SIMULATION = ' + str(ramp_sim))
subprocess.run(['rm', sim_file])
subprocess.run(['cyclus', in_file, '-o', sim_file])
#, stdout=subprocess.PIPE)
#subprocess.run(['cyclus', 'test_py.py', '-o', 'test_py.sqlite'], capture_output=True).stdout

CompletedProcess(args=['cyclus', 'test_py.py', '-o', '../output/test_py.sqlite'], returncode=0)

# Set of Simulations 

List of Simulation Scenarios (24):

If this is done with flat power plus 1% growth in power, doubles to 48 simulations

- EG Scenarios
  1. 01
  2. 01-23
  3. 01-29
- Init Additive Concentration
  1. 100ppt
  2. ???pp?
- Date
  1. beginning of transition
  2. later in transition
- Rate
  1. full availability
  2. ramp up availability

In [19]:
# File Names:
for eg in ['01', '23', '29']:
    for ppx in ['100ppt', '100ppb']:
        for date in ['05yr', '15yr']:
            for rate in ['full', 'ramp']:
                file = eg + '_' + ppx + '_' + date + '_' + rate
                print(file)

01_100ppt_05yr_full
01_100ppt_05yr_ramp
01_100ppt_15yr_full
01_100ppt_15yr_ramp
01_100ppb_05yr_full
01_100ppb_05yr_ramp
01_100ppb_15yr_full
01_100ppb_15yr_ramp
23_100ppt_05yr_full
23_100ppt_05yr_ramp
23_100ppt_15yr_full
23_100ppt_15yr_ramp
23_100ppb_05yr_full
23_100ppb_05yr_ramp
23_100ppb_15yr_full
23_100ppb_15yr_ramp
29_100ppt_05yr_full
29_100ppt_05yr_ramp
29_100ppt_15yr_full
29_100ppt_15yr_ramp
29_100ppb_05yr_full
29_100ppb_05yr_ramp
29_100ppb_15yr_full
29_100ppb_15yr_ramp
